In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from dataclasses import dataclass

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from depsurf import DepKind, Versions
from depsurf.output import (
    bold,
    get_legend_handles_labels,
    get_text_height,
    load_df,
    plot_yticks,
    save_fig,
    label_multiline_text,
)


def plot_legend(fig: plt.Figure):
    handles, labels = get_legend_handles_labels(fig)
    fig.legend(
        handles,
        labels,
        loc="upper center",
        ncol=len(labels) // 2,
        bbox_to_anchor=(0.5, 0.9375),
        frameon=False,
    )


group_labels = {
    Versions.LTS: "Kernel Versions w/ LTS (2 yr)",
    Versions.REGULAR: "Kernel Versions w/ Regular Releases (6 mo)",
    Versions.REV: f"Revisions for Kernel {bold(5.4)}",
}


@dataclass
class SubfigPlotter:
    ax: plt.Axes
    df: pd.DataFrame
    group: Versions
    show_xlabels: bool
    fontsize: int = 8

    @property
    def columns(self):
        return self.df.columns.drop(["Old"])

    @property
    def num_bars(self):
        return len(self.df.index)

    def plot(self):
        self.plot_bar()
        self.plot_xticks()
        plot_yticks(self.ax)
        self.ax.tick_params(axis="both", labelsize=9)
        with plt.rc_context(
            {
                "font.family": "monospace",
                "font.size": self.fontsize,
                # "font.stretch": "condensed",
            }
        ):
            self.plot_val_labels()
            self.plot_top_labels()

    def plot_bar(self, x_pad=0.5):
        bottom = np.zeros(self.num_bars)
        xs = np.arange(self.num_bars)
        for col in self.columns:
            self.ax.bar(xs, self.df[col], label=col, bottom=bottom, color=col.color)
            bottom += self.df[col].to_numpy()

        self.ax.set_xlim(-x_pad, self.num_bars - 1 + x_pad + 0.01)
        self.ax.set_ylim(0, bottom.max() * 1.15)

    def plot_xticks(self):
        xs = np.arange(self.num_bars + 1) - 0.5

        versions = self.group
        labels = self.group.get_labels()
        self.ax.set_xticks(xs, labels)

        if self.group == Versions.REGULAR:
            for x, v in zip(xs, versions):
                if v.lts:
                    self.ax.axvline(x, color="gray", linestyle="--", linewidth=1)

        if not self.show_xlabels:
            self.ax.set_xticklabels([])
        else:
            self.ax.set_xlabel(group_labels[self.group])

    def plot_val_labels(self):
        text_height = get_text_height(self.ax)

        def format_val(val):
            if val > 1000:
                return f"{val / 1000:.2g}k"
            return str(val)

        trans = self.ax.transData
        bottom = np.zeros(self.num_bars)
        for col in self.columns:
            for i, v in enumerate(self.df[col]):
                bar_height = (trans.transform((0, v)) - trans.transform((0, 0)))[1]
                if bar_height < text_height * 0.9:
                    continue
                h = v / 2 + bottom[i]
                self.ax.text(i, h, format_val(v), ha="center", va="center")
            bottom += self.df[col].to_numpy()

    def plot_top_labels(self):
        ymax = self.ax.get_ylim()[1]
        for i, (index, row) in enumerate(self.df.iterrows()):
            old = row["Old"]
            total = sum(row) - old

            added = row["Added"]
            removed = row["Removed"]
            changed = total - added - removed

            if self.group == "rev":

                def format_val(v):
                    return f"{v:0.0f}"

            else:

                def format_val(v):
                    return f"{v / old:0.0%}"

            lines = [
                f"${k}${format_val(v)}" if v != 0 else ""
                for k, v in [
                    (r"\Delta", changed),
                    (r"\minus", removed),
                    (r"\plus", added),
                ]
            ]
            colors = ["darkgreen", "xkcd:dark orange", "blue"]

            label_multiline_text(
                self.ax, i, ymax, lines, colors, fontsize=self.fontsize
            )


fig, axs = plt.subplots(
    3,
    3,
    figsize=(12, 12),
    width_ratios=[4, 16, 5],
    gridspec_kw={"wspace": 0.15, "hspace": 0.075},
)

df = load_df("src").T

for row_idx, (ax_rows, kind) in enumerate(
    zip(axs, [DepKind.FUNC, DepKind.STRUCT, DepKind.TRACEPOINT])
):
    for col_idx, (ax, group) in enumerate(zip(ax_rows, group_labels.keys())):
        df_group = df[kind].loc[group]
        SubfigPlotter(
            ax=ax,
            df=df_group,
            group=group,
            show_xlabels=row_idx == len(axs) - 1,
        ).plot()

        if col_idx == 0:
            ax.set_ylabel(f"Number of {bold(kind.capitalize())} Changes")

plot_legend(fig)
save_fig(fig, "src")

[           pd.py:50 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/src.pkl
[          mpl.py:76 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/src.pdf
